In [1]:
import featuretools as ft

In [2]:
data = ft.demo.load_mock_customer()

In [9]:
customers_df = data['customers']
customers_df.sample(5)

,customer_id,zip_code,join_date,date_of_birth
1,2,13244,2012-04-15 23:31:04,1986-08-18
4,5,60091,2010-07-17 05:27:50,1984-07-28
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
0,1,60091,2011-04-17 10:48:33,1994-07-18


In [8]:
sessions_df = data["sessions"]
sessions_df.sample(5)

,session_id,customer_id,device,session_start
13,14,1,tablet,2014-01-01 03:28:00
6,7,3,tablet,2014-01-01 01:39:40
1,2,5,mobile,2014-01-01 00:17:20
28,29,1,mobile,2014-01-01 07:10:05
24,25,3,desktop,2014-01-01 05:59:40


In [10]:
transactions_df = data['transactions']
transactions_df.sample(5)

,transaction_id,session_id,transaction_time,product_id,amount
140,151,10,2014-01-01 02:31:40,1,68.25
191,269,13,2014-01-01 03:26:55,2,121.07
199,213,14,2014-01-01 03:35:35,4,108.48
454,468,33,2014-01-01 08:11:50,5,46.99
376,43,27,2014-01-01 06:47:20,5,55.26


In [16]:
# entity = (df, index, time_index)
entities = {
    'customers': (customers_df, 'customer_id', 'join_date'),
    'sessions': (sessions_df, 'session_id', 'session_start'),
    'transactions': (transactions_df, 'transaction_id', 'transaction_time')
}

In [17]:
# relation = (parent_entity, parent_variable, child_entity, child_variable)
relationships = [
    ('sessions', 'session_id', 'transactions', 'session_id'),
    ('customers', 'customer_id', 'sessions', 'customer_id')
]


In [18]:
# deep feature synthesis: target: customers
feature_matrix_customers, features_dfs = ft.dfs(entities=entities, relationships=relationships, target_entity='customers')
feature_matrix_customers.sample(5)

,zip_code,COUNT(sessions),NUM_UNIQUE(sessions.device),MODE(sessions.device),SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),...,NUM_UNIQUE(sessions.YEAR(session_start)),MODE(sessions.DAY(session_start)),MODE(sessions.WEEKDAY(session_start)),MODE(sessions.MONTH(session_start)),MODE(sessions.MODE(transactions.product_id)),MODE(sessions.YEAR(session_start)),NUM_UNIQUE(transactions.sessions.customer_id),NUM_UNIQUE(transactions.sessions.device),MODE(transactions.sessions.customer_id),MODE(transactions.sessions.device)
customer_id,,,,,,,,,,,,,,,,,,,,,
2,13244,7,3,desktop,7200.28,37.705178,146.81,0.098259,8.73,77.422366,...,1,1,2,1,3,2014,1,3,2,desktop
4,60091,8,3,mobile,8727.68,45.068765,149.95,-0.036348,5.73,80.070459,...,1,1,2,1,1,2014,1,3,4,mobile
5,60091,6,3,mobile,6349.66,44.095630,149.02,-0.025941,7.55,80.375443,...,1,1,2,1,3,2014,1,3,5,mobile
1,60091,8,3,mobile,9025.62,40.442059,139.43,0.019698,5.81,71.631905,...,1,1,2,1,4,2014,1,3,1,mobile
3,13244,6,3,desktop,6236.62,43.683296,149.15,0.418230,5.89,67.060430,...,1,1,2,1,1,2014,1,3,3,desktop


In [39]:
# deep feature synthesis: target: sessions
feature_matrix_customers, features_dfs = ft.dfs(
    entities=entities, 
    relationships=relationships, 
    target_entity='sessions',
    agg_primitives=['mean', 'max', 'median', 'skew', 'avg_time_between', 'count', 'std', 'n_most_common', 'time_since_last', 'time_since_first', 'trend', 'entropy', 'num_unique'],
    trans_primitives=['year', 'month', 'week', 'weekday', 'day', 'is_weekend', 'time_since_previous', 'cum_mean', 'cum_max', 'cum_min', 'cum_count', 'cum_sum'],
    max_depth=1
)
feature_matrix_customers.sample(5)

,customer_id,device,MEAN(transactions.amount),MAX(transactions.amount),MEDIAN(transactions.amount),SKEW(transactions.amount),AVG_TIME_BETWEEN(transactions.transaction_time),COUNT(transactions),STD(transactions.amount),N_MOST_COMMON(transactions.product_id)[0],...,NUM_UNIQUE(transactions.product_id),YEAR(session_start),MONTH(session_start),WEEK(session_start),WEEKDAY(session_start),DAY(session_start),IS_WEEKEND(session_start),TIME_SINCE_PREVIOUS(session_start),CUM_COUNT(customer_id),customers.zip_code
session_id,,,,,,,,,,,,,,,,,,,,,
8,4,tablet,75.081111,143.85,67.645,0.200676,65.0,18,44.213242,1.0,...,5,2014,1,1,2,1,False,975.0,8,60091
2,5,mobile,74.696000,135.25,75.390,-0.160550,65.0,10,45.893591,5.0,...,5,2014,1,1,2,1,False,1040.0,2,60091
14,1,tablet,85.469167,139.09,99.250,-0.830975,65.0,12,39.825249,4.0,...,5,2014,1,1,2,1,False,780.0,14,60091
9,1,desktop,70.135333,132.72,57.620,0.226337,65.0,15,46.905665,4.0,...,5,2014,1,1,2,1,False,1170.0,9,60091
16,2,desktop,96.581000,138.38,94.255,0.074890,65.0,10,27.839228,4.0,...,5,2014,1,1,2,1,False,520.0,16,13244


In [40]:
features_dfs

[<Feature: customer_id>,
 <Feature: device>,
 <Feature: MEAN(transactions.amount)>,
 <Feature: MAX(transactions.amount)>,
 <Feature: MEDIAN(transactions.amount)>,
 <Feature: SKEW(transactions.amount)>,
 <Feature: AVG_TIME_BETWEEN(transactions.transaction_time)>,
 <Feature: COUNT(transactions)>,
 <Feature: STD(transactions.amount)>,
 <Feature: N_MOST_COMMON(transactions.product_id)>,
 <Feature: TIME_SINCE_LAST(transactions.transaction_time)>,
 <Feature: TIME_SINCE_FIRST(transactions.transaction_time)>,
 <Feature: TREND(transactions.amount, transaction_time)>,
 <Feature: ENTROPY(transactions.product_id)>,
 <Feature: NUM_UNIQUE(transactions.product_id)>,
 <Feature: YEAR(session_start)>,
 <Feature: MONTH(session_start)>,
 <Feature: WEEK(session_start)>,
 <Feature: WEEKDAY(session_start)>,
 <Feature: DAY(session_start)>,
 <Feature: IS_WEEKEND(session_start)>,
 <Feature: TIME_SINCE_PREVIOUS(session_start)>,
 <Feature: CUM_COUNT(customer_id)>,
 <Feature: customers.zip_code>]